In [1]:
import torch
from saris.drl.agents import sac
from tensordict import TensorDict, from_module, from_modules, TensorDictBase

In [2]:
checkpoint_path = "/home/hieule/research/saris/local_assets/logs/SAC__L_shape_static__wireless-sigmap-v0__trained/model_1200.pth"
ob_dim = 84
ac_dim = 27

In [3]:
qf1 = sac.SoftQNetwork(ob_dim, ac_dim).to("cuda")
qf2 = sac.SoftQNetwork(ob_dim, ac_dim).to("cuda")
qnet_params = from_modules(qf1, qf2, as_module=True)
qnet_target_params = qnet_params.data.clone()

In [4]:
checkpoint = torch.load(checkpoint_path, weights_only=False)

In [5]:
qnet_params_state_dict = qnet_params.state_dict()
qnet_target_params_state_dict = qnet_target_params.state_dict()

for layer_name, layer_params in checkpoint['qnet_params'].items():
    qnet_params_state_dict[layer_name] = layer_params
    
for layer_name, layer_params in checkpoint['qnet_target_params'].items():
    qnet_target_params_state_dict[layer_name] = layer_params

In [6]:
qnet_params.load_state_dict(qnet_params_state_dict)
qnet_target_params = TensorDict(qnet_target_params)

In [7]:
qnet_target_params.load_state_dict(qnet_target_params_state_dict)
qnet_target_params = TensorDict(qnet_target_params)

In [8]:
checkpoint['qnet_params'] = qnet_params.state_dict()
checkpoint['qnet_target_params'] = qnet_target_params.state_dict()

torch.save(checkpoint, "/home/hieule/research/saris/local_assets/logs/SAC__L_shape_static__wireless-sigmap-v0__trained/model_trained.pth")